Aluno: Gabriel Guimarães Almeida de Castro  - 150126425 - Segurança Computacional - 10/05/2021



# Criptografia RSA 1024bits

O sistema de criptografia RSA é um dos primeiros sistemas de chave pública, e o nome são as iniciais dos sobrenomes dos criadores (Rivest-Shamir-Adleman). A dificuldade para descriptografar algo criptografado por RSA se deve ao tempo computacional necessário para fatorar números primos grandes.

Sendo assim, para criptografar e descriptografar não é necessário muito tempo quando se conhece as chaves. Mas quando não se conhece o tempo necessário pode ser inviável.

Apesar de toda a segurança, devido ao uso de blocos no algorítmo mostrado abaixo, é possível descriptografar mais rapidamente devido a frequência com que certos blocos aparece. Algo que pode aumentar a dificuldade é usar números junto.

A biblioteca pickle serve para converter arquivos do formato da RAM para um fluxo de caracteres e também o inverso

In [96]:
import math
import random
import pickle as pickle

# Teste de primalidade usando Miller-Rabin

Para encontrar números primos grande de forma mais rápida usamos o teste de Rabin Miller que é um teste probabilístico que indica um provável número primo. 

O teorema dos números primos diz que um bom número de testes é  ln(2^k) / 2. Pois para uma chave de 1024 bits n ln(2^1024) = 710, mas como os números primos são ímpares com excessão do 2, então 710/2 = 355 

In [97]:
def miller_rabin(num, key_size=1024):
    test_num = round(math.log(pow(2, key_size))/2)
    s = num - 1
    t = 0
   
    while s % 2 == 0:
        s = s // 2
        t += 1
    for trials in range(test_num):
        a = random.randrange(2, num - 1)
        v = pow(a, s, num)
        if v != 1:
            i = 0
            while v != (num - 1):
                if i == t - 1:
                    return False
                else:
                    i = i + 1
                    v = (v ** 2) % num
        return True
def is_prime(num, keysize):
    if (num < 2):
        return False
    lowPrimes = [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 
   67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 
   157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 
   251, 257, 263, 269, 271, 277, 281, 283, 293, 307, 311, 313,317, 331, 337, 347, 349, 
   353, 359, 367, 373, 379, 383, 389, 397, 401, 409, 419, 421, 431, 433, 439, 443, 449, 
   457, 461, 463, 467, 479, 487, 491, 499, 503, 509, 521, 523, 541, 547, 557, 563, 569, 
   571, 577, 587, 593, 599, 601, 607, 613, 617, 619, 631, 641, 643, 647, 653, 659, 661, 
   673, 677, 683, 691, 701, 709, 719, 727, 733, 739, 743, 751, 757, 761, 769, 773, 787, 
   797, 809, 811, 821, 823, 827, 829, 839, 853, 857, 859, 863, 877, 881, 883, 887, 907, 
   911, 919, 929, 937, 941, 947, 953, 967, 971, 977, 983, 991, 997]
	
    if num in lowPrimes:
        return True
    for prime in lowPrimes:
        if (num % prime == 0):
            return False
    return miller_rabin(num, keysize)


## Funções básicas

Segue abaixo algumas funções matemáticas básicas como o algorítmo euclidiano para máximo divisor comum, o algorítmo euclidiano estendido e a função totiente de euler.

In [120]:
# find GCD    
def gcd(a, b):
    # Algorítmo Euclidiano de máximo divisor comum
    while a != 0:
        a, b = b % a, a
    return b


In [128]:
# EAlgorítmo Euclidiano extendido
def mod_inv(a, b):
    mod = b
    y = 0
    x = 1
    if b == 1:
        return 0
    
    
    while a > 1:
        k = a // b
        temp = b
        
        b = a % b
        a = temp
        temp = y
        
        y = x - k * y
        x = temp
    if x < 0:
        x = x + mod
        return x

## Função que gera o número primo

In [121]:
def generate_prime_num(keysize=1024):
    while True:
        num = random.randrange(2**(keysize-1), 2**(keysize))
        if is_prime(num, keysize):
            return num

## Função que o gera as chaves

In [122]:
def generate_rsa_key():
    print('Gerando chaves')
    prime_a = generate_prime_num()
    prime_b = generate_prime_num()
    num = prime_a * prime_b
    phi = (prime_a-1)*(prime_b-1)
    e = random.randrange(1, phi)
    while gcd(e, phi) != 1:
        e = random.randrange(1, phi)
    d = mod_inv(e, phi)
    return ((d, num), (e, num))


## Função criptografar

Função que criptografa usando a chave pública.

In [123]:
def encrypt(public_key, msg):
    e, num = public_key
    encrypted_msg = []
    
    for i in msg:
        temp = ord(i)
        encrypted_msg.append(pow(temp, e, num))
    return encrypted_msg

## Função descriptografar

Função que descriptografa a mensagem usando a chave privada

In [124]:
def decrypt(private_key, encrypted_msg):
    d, num = private_key
    
    plain_text = ''
    
    for i in encrypted_msg:
        temp = pow(i, d, num)
        plain_text = plain_text + str(chr(temp))
        
    return plain_text

## Criando as chaves
Cria as chaves e guarda em um arquivo como um binário.

In [125]:
if __name__ == "__main__":
    path_priv = "./files/private_key.txt"
    path_pub = "./files/public_key.txt"
    
    private_key, public_key = generate_rsa_key()
    print('chave pública:', public_key)
    print('chave privada:', private_key)
    
    with open(path_priv, 'wb') as file_priv:
        pickle.dump(private_key, file_priv)
    with open(path_pub, 'wb') as file_pub:
        pickle.dump(public_key, file_pub)

Gerando chaves
chave pública: (3157162338733091999544672817847592569528939244038416034996088713805739585203624644299246965289799269164989634643492669873015802151147590147716351908889202477294251507927816904160208795940865307212877365488880211890129990373399545626190690689144025600064119884401195629906007009038068197623664967758096350463409544089197631774492306359092655519435129779504135608502461559963715585650166530409974165167050979397286100172791307228527829075191239077700908989414132666699056972903536788562966884359475663211602286725480995891801494743694576801294389398718215832158434236506296119207569498510110274495373600766440479521719, 920680859285249919669944002268966886377512362770323672889443344946044922628298197067601795400978776008542643226192265951021305463764835236340207556462266939128757577026156654597106070776345576339862110372698005929321537060098807345977158253101778469944505106521848534423474273845349588131647131812681929196962075402224958395925971274485363706679846668

## Criptografa

Criptografa e guarda arquivo com a mensagem em binário

In [126]:
path_text = './files/plaintext.txt'
path_encrypted = './files/encrypted_msg.txt'

with open(path_pub, 'rb') as file_pub:
    with open(path_text, 'r') as file_text:
        cipher_text = encrypt(pickle.load(file_pub), file_text.read())
        
        print("Texto criptografado:", cipher_text)
        
    with open(path_encrypted, 'wb') as file_encrypted:
        pickle.dump(cipher_text, file_encrypted)

Texto criptografado: [...]


## Descriptografando

Descriptografa arquivo e mostra o texto.

In [127]:
with open(path_priv, 'rb') as file_priv:
    with open(path_encrypted, 'rb') as file_encrypted:
        plain_text = decrypt(pickle.load(file_priv), pickle.load(file_encrypted))
        
        print("Texto descriptografado:", plain_text)

Texto descriptografado: O teste Miller-Rabin (por Gary Miller e Michael Rabin) é um teste probabilístico da primitividade de um dado número n. Se um número n não passar pelo teste, n com certeza é um número composto (ou seja, não-primo). Se o número passar no teste, ele é primo, com uma probabilidade P >= 0,75, sendo que P denomina o conjunto de todos números primos.



## Referências
https://medium.com/@prudywsh/how-to-generate-big-prime-numbers-miller-rabin-49e6e6af32fb

https://github.com/jchen2186/rsa-implementation

https://www.geeksforgeeks.org/euclidean-algorithms-basic-and-extended/
    
https://www.youtube.com/watch?v=vbm0zEv9eP4

https://pt.wikipedia.org/wiki/Teste_de_primalidade_de_Miller-Rabin

https://pt.wikipedia.org/wiki/RSA_(sistema_criptogr%C3%A1fico)